In [1]:
import numpy as np
#myqlm imports
from qat.lang.AQASM import Program, H, RX, RY, RZ, Z, CNOT
from qat.lang.AQASM import *
from qat.qpus import get_default_qpu

## Quantum feature map

Let the attributes be $x \in X$, where $X$ represents a non-linearly separable dataset. In order to map the data to a space where it becomes linearly separable by a hyperplane, kernel methods are used. From the mathematical definition of a kernel function, we have that $K(x,z) = \langle \phi(x), \phi(z) \rangle $, $x$ and $z$ being n-dimensional vectors. The $\phi(x)$ function maps $x: \mathbf{R}^n \rightarrow \mathbf{R}^m$, where $m$ is usually much larger than $n$. A quantum feature map is a function that strictly plays the same role such that the mapping is done using an \textit{n}-qubit operator so the result is a vector that lives in the higher-order Hilbert space, $\mathcal{H}$. So $x: \mathbf{R}^n \rightarrow \mathcal{H}$. The quantum feature map 
\begin{equation}
    \mathcal{U}_{\Phi(x)}|0\rangle^{\otimes n} = |\Phi(x)\rangle
\end{equation}
has been shown to play an important role toward the quantum advantage on supervised learning tasks on quantum computers for specifics data sets. In addition to quantum ZZ feature map used in some references, an another operator could be 

$$ \mathrm{U}_{\Phi(x)} = \bigotimes_{i=0}^{n}RX(x_i) $$

since $x$ should be normalized using min-max approach such that $x \in \left[0, 2\pi\right]$. Such quantum feature map does not produce any effect in circuit depth, remaining $d = \mathcal{O}(1)$. In the other hand, the number o qubits scales linearly with the features vector size, remaining $n = \mathcal{O}(|X|)$.



In [2]:
def data_embedding(x):
    """
    Args
        x: a np.array containing normalized feature vector;
    Outpu
        emb: a quantum circuit that encodes the data;
        
    """

    emb = QRoutine()
    wires = emb.new_wires(len(x))
    with emb.compute():
        for i, wire in enumerate(wires):
            RX(x[i])(wire)
    
    return emb

## Tunable ansatz

The parametrized quantum circuit used in this quantum neural network is a two-local heuristic pattern using a full entangled quantum circuit. Here, the CNOT's set will be the operator $W$. The mathematical representation of a multi-layer of the ansatz is given by

$$P(\vec{\theta}) = \Pi_{j=1}^{L}\bigotimes_{i=0}^{n}RY(\theta_i)\bigotimes_{i=0}^{n}RZ(\theta_i) (W) \bigotimes_{i=0}^{n}RY(\theta_{i+n})\bigotimes_{i=0}^{n}RZ(\theta_{i+n})$$

where $\vec{\theta}$ is the parameter vector analogous to the weights in a classical neural network. Such parameters are trained via classical optimizers.

In [3]:
def ansatz(params, feature_len, num_layers):
    """
    Args
        params: np.array with tunable parameters in the ansatz;
        feature_len: a integer which is the size of feature vector (atributes);
        num_layers: a integer which is the number of layers;
    Outpu
        pcirc: a quantum circuit which is the ansatz;
        
    """  

    pcirc = QRoutine()
    wires = pcirc.new_wires(feature_len)
    #writing quantum circuit for the ansatz
    for layer in range(num_layers):
        with pcirc.compute():
            for i, wire in enumerate(wires):
                RY(params[i + layer*feature_len])(wire)
                RZ(params[i + layer*feature_len])(wire)          
            if layer == num_layers:
                break
            
            #circular entanglement
            CNOT(wires[0], wires[1])
            CNOT(wires[0], wires[2])
            CNOT(wires[0], wires[3])
            CNOT(wires[1], wires[2])
            CNOT(wires[1], wires[3])            
            CNOT(wires[2], wires[3])
            
    return pcirc